<a href="https://colab.research.google.com/github/yandexdataschool/MLatImperial2022/blob/master/Seminars/lab_05_Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hello, pytorch

![img](https://upload.wikimedia.org/wikipedia/commons/9/96/Pytorch_logo.png)


> **!!!** If you're running this notebook on Colab, make sure you enable the GPU support by going to `'Edit'->'Notebook settings'` and setting `'Hardware accelerator'` to `'GPU'`.

[PyTorch](http://pytorch.org/) is one of the most commonly used deep learning frameworks. Pytorch tools are notable for their ability to compute gradients automatically and do operations on GPU, which can be by orders of magnitude faster than running on CPU.

In pytorch you can compute outputs on the fly without pre-declaring anything, and the code looks (almost) exactly as in pure numpy. Due to this simplicity, PyTorch is going to be our weapon of choice for this course.

We'll start from using the low-level core of PyTorch, and then try out some high-level features.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import torch
print(torch.__version__)

In [ ]:
# numpy operations
x = np.arange(16).reshape(4, 4)

print(f"X :\n{x}\n")
print(f"X.shape : {x.shape}\n")
print(f"X.shape + dummy dim: {x[None, :, :].shape}\n") # or np.newaxis
print(f"add 5 :\n{x + 5}\n")
print(f"X*X^T  :\n{np.dot(x, x.T)}\n")
print(f"mean along cols :\n{x.mean(axis=1)}\n")
print(f"cumsum of cols :\n{np.cumsum(x, axis=1)}\n")

In [ ]:
# pytorch operations
x = torch.arange(16.0).reshape(4, 4) # What if we pass 16 instead of 16.0?

print(f"X :\n{x}\n")
print(f"X.shape : {x.shape}\n")
print(f"X.shape + dummy dim: {x.unsqueeze(0).shape}\n")
print(f"add 5 :\n{x + 5}")
print(f"X*X^T  :\n{torch.matmul(x, x.T)}\n") # torch.dot assumes arguments are 1D vectors (does inner product)
print(f"mean along cols :\n{x.mean(dim=1)}\n")
print(f"cumsum of cols :\n{x.cumsum(dim=1)}\n")

### Converting numpy <--> pytorch

In [ ]:
X_numpy = np.array([1, 2, 3])
X_torch = torch.from_numpy(X_numpy)
X_numpy_again = X_torch.numpy()

X_numpy, X_torch, X_numpy_again

### Sending tensors to GPU

In [ ]:
print(f"cuda is available: {torch.cuda.is_available()}")
X_gpu = X_torch.cuda() # .to(device), device = 'cpu' or 'cuda'
print(X_gpu)

# Let's convert to numpy
X_gpu.numpy()

In [ ]:
print(X_gpu.cpu().numpy(), X_gpu.device, X_torch.device)

### Types in pytorch

In [ ]:
values = [-1, 2, -3, 4]

X_float = torch.tensor(values, dtype=torch.float)
X_float2 = torch.tensor(values).float()

X_long = torch.tensor(values, dtype=torch.long) # torch.int is int32, torch.long is int64
X_long2 = torch.tensor(values).long()


print(X_float, X_float2)
print(X_long, X_long2)

In [ ]:
print(X_float.long())

## NumPy and Pytorch

As you can notice, pytorch allows you to write code much in the same way you did with numpy. But Numpy does not have GPU support, autogradient calculations and many other tools you need for DL. But Pytorch does!

You may also see a few new method names in Pytorch, but mainly they are similar to Numpy. Get excited!

![img](http://i0.kym-cdn.com/entries/icons/original/000/017/886/download.jpg)


To help you acclimatize, there's a [table](https://github.com/torch/torch7/wiki/Torch-for-Numpy-users) covering most new things. There's also a neat [documentation page](http://pytorch.org/docs/master/).

Finally, if you're stuck with a technical problem, we recommend searching [pytorch forums](https://discuss.pytorch.org/). Or just googling, which usually works just as efficiently. 

If you feel like you almost give up, remember two things: **GPU** and **gradients for free**. Besides you can always jump back to numpy with x.numpy().

### Warmup: analytical solution to Ridge regression
$$
X \in \mathbb{R}^{N \times D}, \ y \in \mathbb{R}^{N}, \theta \in \mathbb{R}^{D}\\
\mathcal{L}_{\theta} = \sum_{i=1}^{N} (<X_{i}, \theta>  - y_{i})^{2} + \lambda \sum_{i=1}^{D} \theta_{i}^{2} \rightarrow \min_{\theta}
$$
Stationary condition:
$$
\nabla_{\theta}\mathcal{L}_{\theta} = 2\sum_{i=1}^{N} X_{i}(<X_{i}, \theta>  - y_{i}) + 2\lambda\theta = \overline{0}
$$
Stationary condition in matrix notation:
$$
\nabla_{\theta}\mathcal{L}_{\theta} = 2 X^{T}(X\theta  - y) + 2\lambda\theta = \overline{0} \\
$$
Solution:
$$
(X^{T}X + \lambda\mathrm{I}) \theta = X^{T}y\\
\theta = (X^{T}X + \lambda\mathrm{I})^{-1}X^{T}y
$$

In [ ]:
from sklearn.linear_model import Ridge

N, D = 100, 10

X = torch.randn((N, D))
y = torch.randn((N, 1))
lambda_ = 1.0

inv_term = <YOUR_CODE> # use torch.linalg.inv
theta = <YOUR_CODE>

# Compare our solution with sklearn Ridge solver
model = Ridge(lambda_, fit_intercept=False)
model.fit(X.numpy(), y.numpy())

#Check if solutions are close
assert np.allclose(theta.numpy(), model.coef_.T, atol=1e-6), "Theta is not a solution"
print("Yes, they are close!")

## Automatic gradients

Any self-respecting DL framework must do your backprop for you. Torch handles this through `requires_grad` parameter when creating tensors.

The general pipeline looks like this:
* You create ```a = torch.tensor(data, requires_grad=True)```
* You define some differentiable `loss = whatever(a)`
* Call `loss.backward()`
* Gradients are now available as ```a.grad```

**Here's an example:** let's fit a linear regression on Boston house prices

### Calculate gradients

In [ ]:
!pip install torchviz
from torchviz import make_dot

In [ ]:
a = torch.tensor([2.0], requires_grad=True) # What if dtype is integer
b = torch.tensor([10.0], requires_grad=False)

c = b * a**2
# analytically:
#   dc/da = 2 * a * b = 40
#   dc/db = a * a = 4
print(a.grad, b.grad)
c.backward()
print(a.grad, b.grad)

$$
c = mul(b, pow(a, 2))\\
dc = dmul(b, pow(a, 2)) = (pow(a, 2))db + b * dpow(a, 2) = (pow(a, 2))db + (2 * b * a)da \\
\dfrac{dc}{da} = \{\dfrac{db}{da} = 0, \dfrac{da}{da} = 1\} = 2 * a * b \\ 
\dfrac{dc}{db} = \{\dfrac{da}{db} = 0, \dfrac{db}{db} = 1\} = pow(a, 2) = a * a \\
$$

In [ ]:
make_dot(c)

In [ ]:
a = torch.tensor([2.0]*10, requires_grad=True)
b = torch.tensor([10.0]*10, requires_grad=True)

c = b * a**2
# analytically:
#   dc/da = 2 * a * b = 40
#   dc/db = a * a = 4
c.backward(torch.ones(c.shape[0])) # this way it will compute element-wise grad([1.0*c, 1.0*c, ..., 1.0*c]), same shape as c
print(a.grad, b.grad)

In [ ]:
make_dot(c)

### Detaching

In [ ]:
print(c)
print(c.detach().numpy())

## Boston dataset

In [ ]:
from sklearn.datasets import load_boston
boston = load_boston()
# Let's have a look at the last feature:

plt.figure(figsize=(8, 8))
plt.scatter(boston.data[:, -1], boston.target);
plt.xlabel('Last feature')
plt.ylabel('Target')
plt.show()

In [ ]:
# Defining the parameters of our model:
w = torch.zeros((1, ), requires_grad=True) # float by default
b = torch.zeros((1, ), requires_grad=True)

# Converting our data to torch tensors:
from sklearn.preprocessing import StandardScaler

X = torch.from_numpy(StandardScaler().fit_transform(boston.data)[:,-1]) # select last feature
y = torch.from_numpy(boston.target)

print(f"X.shape : {X.shape}\ny.shape: {y.shape}\nw.shape : {w.shape}\nb.shape : {b.shape}\n")

# prediction:
y_pred = X*w + b
# loss:
loss = ((y_pred - y)**2).mean()
print("Loss is:", loss)

# Compute gradients:
loss.backward()

The gradients are now stored in `.grad` of a variable.

In [ ]:
print("dL/dw =", w.grad)
print("dL/db =", b.grad)

If you compute gradients from multiple losses, the gradients will add up at variables, therefore you usually want to **zero the gradients** between iteratons.

In [ ]:
from IPython.display import clear_output

# let's keep the history of loss values at 
# each step
losses = []
lr = 0.05 # learning rate

w_grads = []
b_grads = []

# Gradient descent:
for i in range(50):
    # Calculate current prediction and loss:
    y_pred = w * X  + b
    loss = ((y_pred - y)**2).mean()
    losses.append(loss.item())
    #                   ^^^
    # The '.item' call returns the value of this 
    # tensor as a standard Python number (this only works
    # for tensors with one element).

    # Compute the gradients
    loss.backward()

    w_grads.append(w.grad.data.item())
    b_grads.append(b.grad.data.item())
    with torch.no_grad():
        # ^^^ this `with` statement ensures that
        #     automatic gradients will not be computed
        #     for whatever is written in the current block:
        w -= lr * w.grad
        b -= lr * b.grad


    # manually zero the gradients
    w.grad.zero_()
    b.grad.zero_()

    # the rest of code is just bells and whistles
    clear_output(True)
    plt.figure(figsize=(14, 5))

    # We'll draw the data points and fit result on the left subplot
    plt.subplot(1, 2, 1)
    plt.title("data")
    plt.scatter(X.numpy(), y.numpy())
    plt.scatter(X.numpy(), y_pred.detach().numpy(),
                color='orange', linewidth=5)
    # The '.detach()' call is needed here to create a
    # copy of 'y_pred' that does not require gradients
    # (otherwize it's impossible to get a numpy
    # representation).

    # We'll draw the loss (as the function of gradient descent
    # step) on the right subplot
    plt.subplot(1, 2, 2)
    plt.title("loss")
    plt.plot(losses)
    plt.xlabel('step')

    plt.show()

    print("loss = ", loss.detach().numpy())

In [ ]:
print("W grads: ", w_grads[:5])
print("b grads: ", b_grads[:5])

try implementing and writing some nonlinear regression. You can try quadratic features or some trigonometry, or a simple neural network. The only difference is that now you have more variables and a more complicated `y_pred`. 

In [ ]:
from IPython.display import clear_output

# let's keep the history of loss values at 
# each step
losses = []
lr = 0.05

powers = range(0, 3)
w = torch.zeros(len(powers), requires_grad=True, dtype=torch.float64)

# Implement feature transformation: X_p = [X^0, X^1, X^2] <-- column-wise
X_p = <YOUR_CODE> # power of 0, 1, 2

# Gradient descent:
for i in range(100):
    # Calculate current prediction and loss:
    y_pred = <YOUR_CODE>
    loss = ((y_pred - y)**2).mean()
    losses.append(loss.item())
    #                   ^^^
    # The '.item' call returns the value of this 
    # tensor as a standard Python number (this only works
    # for tensors with one element).
  
    # Compute the gradients
    loss.backward()

    with torch.no_grad():
    # ^^^ this `with` statement ensures that
    #     automatic gradients will not be computed
    #     for whatever is written in the current block:
        w -= lr  * w.grad


    # manually zero the gradients
    w.grad.zero_()

    # the rest of code is just bells and whistles
    if (i + 1) % 5 == 0:
        clear_output(True)
        plt.figure(figsize=(14, 5))

        # We'll draw the data points and fit result on the left subplot
        plt.subplot(1, 2, 1)
        plt.title("data")
        plt.scatter(X.numpy(), y.numpy())
        plt.scatter(X.numpy(), y_pred.detach().numpy(),
                    color='orange', linewidth=5)
        # The '.detach()' call is needed here to create a
        # copy of 'y_pred' that does not require gradients
        # (otherwize it's impossible to get a numpy
        # representation).

        # We'll draw the loss (as the function of gradient descent
        # step) on the right subplot
        plt.subplot(1, 2, 2)
        plt.title("loss")
        plt.plot(losses)
        plt.xlabel('step')

        plt.show()

        print("loss = ", loss.detach().numpy())

## High-level PyTorch

We're now going to teach the computer to identify some clothes. Exciting, isn't it? The dataset we'll use for that is called [Fashion-MNIST](https://github.com/zalandoresearch/fashion-mnist). Here's an excerpt from their website:

> *Fashion-MNIST is a dataset of Zalando's article images—consisting of a training set of 60,000 examples and a test set of 10,000 examples. Each example is a 28x28 grayscale image, associated with a label from 10 classes. We intend Fashion-MNIST to serve as a direct drop-in replacement for the original MNIST dataset for benchmarking machine learning algorithms. It shares the same image size and structure of training and testing splits.*

So, if you're familiar with the original [MNIST](http://yann.lecun.com/exdb/mnist/) dataset (handwritten digits), this is kinda the same, but about clothes.

In [ ]:
#!rm -r FashionMNIST

PyTorch has a module dedicated to retrieving datasets, so let's use it:

In [ ]:
from torchvision.datasets import FashionMNIST

In [ ]:
# Getting the train and test parts of the dataset
data_train = FashionMNIST("FashionMNIST/",
                          download=True,
                          train=True)

data_test = FashionMNIST("FashionMNIST/",
                          download=True,
                          train=False)

# In fact, it's already stored as torch tensor, but we'll need
# to work with the numpy representation, so let's do the convertion:
X_train = data_train.train_data.numpy()
y_train = data_train.train_labels.numpy()

X_test = data_test.test_data.numpy()
y_test = data_test.test_labels.numpy()

The datasets consists of images belonging to one out of 10 classes:

| Label | Description | | Label | Description |
| ---       
| 0        | T-shirt/top   || 5        | Sandal         |
| 1        | Trouser        || 6        | Shirt             |
| 2        | Pullover       || 7        | Sneaker       |
| 3        | Dress           || 8        | Bag              |
| 4        | Coat             || 9        | Ankle boot  |






Let's check out the shapes of our arrays:

In [ ]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

And now we are going to plot images from specific categories

In [ ]:
categories = [
    X_train[y_train == i]
    for i in range(10)
]

ten_of_each = np.array([c[:10] for c in categories])
ten_of_each = np.transpose(ten_of_each, (0, 2, 1, 3)).reshape(280, 280)

plt.figure(figsize=(10, 10))
plt.imshow(ten_of_each, cmap='Greys')
plt.axis('off');

Ok, so let's start writing our first deep learning model! For that we'll need 3 things:
  - a **loss** function that takes predicted class scores and true labels, and outputs a measure of how wrong we are in our predictions
  - a **function** that converts the input features to class scores (this will be represented by a neural network)
  - an **optimization algorithm** that'll vary the parameters of our model to minimize the loss

### Loss function

Since we are facing a classification problem, **cross-entropy** (aka negative log likelihood) is a good choice for a loss function:
$$\mathscr{L} = -\frac{1}{|D|}\sum_{c\in C,\,i\in D}p^{\text{true}}_{c,i}\cdot log\,\hat{p}^{\text{predicted}}_{c,i},$$
where $C$ is the set of all classes and $D$ is the set of all objects.

<font color='red'>Question:</font> *What is $p^{\text{true}}_{c,i}$ equal to for a given object $i$ of class $c'$?*
<font color='red'>Question (2):</font> *What will $\mathscr{L}$ look like for a binary classification case?*

Since our loss function takes in *probabilities*, we have to make sure they add up to 1 for any given object. In general case, the output of a neural net can be anything, but there's a commonly used activation function that automatically normalizes the outputs to add up to 1:

$$f(x_i) = \frac{e^{x_i}}{\sum_j e^{x_j}}$$

This is the *softmax function*, and we'll use it as the activation for the very last layer.

Let's start by trying to implement this function (in torch):

In [ ]:
def softmax(x):
    <YOUR_CODE>

Once you're done, run the cell below to check whether your function passes automatic checks.

In [ ]:
# Creating some simple input
dummy_input_1 = torch.tensor([0.5, -10., 4., 1.333])
dummy_input_2 = 100 * dummy_input_1

# Computing the output with your implementation
dummy_result_1 = softmax(dummy_input_1)
dummy_result_2 = softmax(dummy_input_2)

# Checking the output type
assert isinstance(dummy_result_1, torch.Tensor), \
       "Your function should return a torch tensor"

# Correct outputs (for the automatic checks):
dummy_correct_result_1 = torch.tensor([2.7460692e-02,
                                   7.5616998e-07,
                                   9.0937322e-01,
                                   6.3165329e-02])
dummy_correct_result_2 = torch.tensor([0., 0., 1., 0.])

# Doing the checks:
assert torch.allclose(
            dummy_result_1,
            dummy_correct_result_1
       ), "Test failed!\n" \
       "Looks like something is wrong " \
       "with your softmax implementation.\nCorrect result is:\n{}\n" \
       "You got:\n{}".format(dummy_correct_result_1, dummy_result_1)

assert torch.isfinite(dummy_result_2).all(), \
       "Test failed!\n" \
       "Looks like your implementation is unstable to large inputs.\n" \
       "You got:\n{}\n" \
       "Can you think of a way of fixing it?".format(dummy_result_2)

assert torch.allclose(
            dummy_result_2,
            dummy_correct_result_2
       ), "Test failed!\n" \
       "Looks like something is wrong " \
       "with your softmax implementation.\nCorrect result is:\n{}\n" \
       "You got:\n{}".format(dummy_correct_result_2, dummy_result_2)

# Printout for the case when all is good.
print("Tests passed!")

Were you able to regularize your softmax implementation?

In fact, since we are combining crossentropy and softmax, we'll always take a `log` of the softmax output (see formulas above). This means one more level of numerical regularisation of the final expression can be done. This has already been implemented in the high-level PyTorch `nn` module within the `torch.nn.CrossEntropyLoss` class.

Let's use it to create our loss function as follows:

In [ ]:
# Defining the loss function:
loss_function = torch.nn.CrossEntropyLoss()

The `loss_function` now takes two arguments as its input: score predictions (output of the last layer, without activation) and the correct labels:

$$\mathscr{L}(x, c) = -\frac{1}{|D|}\sum_{i\in D}log\,\frac{e^{x_i^{c_i}}}{\sum_{j\in C}e^{x_i^j}},$$

where $x_i^j$ is the score for the $i$-th object to belong the $j$-th class, $c_i$ is the true class for $i$-th object; $C$ is the set of all classes and $D$ is the set of all objects

Below is an example of how `loss_function` can now be used.

**Task:** try changing the values of `dummy_x` to make the resulting loss even smaller (min loss value is 0.0).

In [ ]:
# Example usage (play around with this code)

# Think of dummy_x as of output of the last layer of our neural net.
# For example, dummy_x below corresponds to a 3-class classification of
# 5 objects:
dummy_x = torch.tensor(
    [[9., -5., -10.], # here we're almost sure it's class 0
     [-3.,  8.,   9.], # here we think it's either class 1 or 2, etc
     [-3., 15., -10.],
     [ 4.,  5.,  -3.],
     [-3., 15., -10.]]
)
# And these are the answers (vector of class ids):
dummy_y = torch.tensor([0, 2, 1, 1, 1])

print(loss_function(dummy_x, dummy_y))

### Neural net

Now that we have a loss function, it's time to build our network. Again, there's a bunch of ready to use classes in the `torch.nn` module.

Let's start from something really simple - a model without any hidden layers. For such a case we should only have 1 input layer, 1 output layer and a set of linear connections between them:

$$\text{output} = XW_{linear} + b_{linear}$$\\

$$
X \in \mathbb{R}^{N \ \times \ \text{input_dim}}\\
W_{linear}\in \mathbb{R}^{\text{input_dim}\  \times \ \text{output_dim}}\\
b_{linear} \in \mathbb{R}^{\text{output_dim}}\\
$$

We'll use the `torch.nn.Linear` class to define these connections:

In [ ]:
input_dim = 28 * 28 # number of pixels per image
output_dim = 10 # number of classes
model = torch.nn.Linear(in_features=input_dim,
                        out_features=output_dim).cuda()

Here's an example of how it works:

In [ ]:
# You can think of dummy_input as of 5 images with all pixels black:
dummy_input = torch.zeros((5, 28 * 28)).cuda()
# Then our output is going to be the 10 class scores for each of the images:
dummy_output = model(dummy_input)

# Let's have a look at their shape:
print(dummy_input.shape)
print(dummy_output.shape)

**Question:** how many parameters does this model have?

One can iterate through the `model`'s parameters with it's `.parameters()` method. Try using it to calculate the number of (scalar) parameters:

In [ ]:
# calculate the number of (scalar) parameters:
n_parameters = 0
for parameter in model.parameters():
    n_parameters += parameter.reshape(-1).shape[0]

print(n_parameters)

### Building dataset and Input preprocessing

So far our data is held as numpy arrays of unsigned byte type, i.e. it lies within a range from 0 to 255. Also, the shape of our input is 3-dimensional (num_images, height, width), while our `model` takes 2-dimensional "arrays of 1-dimensional images" (num_images, height * width).

In [ ]:
print(X_train.shape)
print(X_train.dtype)
print(X_train.min(), X_train.max())

We have to create special dataset class that converts data to `torch` tensors, normalizes it to $[0, 1]$ interval and reshapes the input.

In [ ]:
from torch.utils.data import Dataset, DataLoader

class FashionMNISTDataset(Dataset):
    def __init__(self, X, y=None):
        self.X, self.y = self.preprocess_data(X, y)
        
    def preprocess_data(self, X, y):
        X_preprocessed = torch.tensor(X / 255.,
                                    dtype=torch.float).reshape(-1, 28 * 28).cuda()
        
        if (y is None):
            return X_preprocessed, None
        
        return X_preprocessed, torch.tensor(y).cuda()
        
    def __len__(self):
        return self.X.shape[0]
    
    def __getitem__(self, idx):
        if (self.y is None):
            return self.X[idx]
        
        return self.X[idx], self.y[idx]

In [ ]:
BATCH_SIZE = 128

ds_train = FashionMNISTDataset(X_train, y_train)
ds_test = FashionMNISTDataset(X_test, y_test)

dl_train = DataLoader(ds_train, batch_size = BATCH_SIZE, shuffle=True)
dl_test = DataLoader(ds_test, batch_size = BATCH_SIZE, shuffle=False)
y_test = torch.tensor(y_test).cuda()

The simplest way to check this function is to use it to calculate loss:

In [ ]:
sample_X, sample_y = next(iter(dl_train))

assert sample_X.min().item() >= 0 and \
       sample_X.max().item() <= 1, "Make sure your input is >= 0 and <= 1"

class_scores = model(sample_X)
print("Loss is:", loss_function(class_scores, sample_y).item())

### Optimizer

Now, to train our network we need an optimization algorithm. 

Actually, we already implemented one ourselves: stochastic gradient descent (from the *Automatic gradients* section of this notebook). Back then we manually updated our model's parameters by adding a `-learning_rate * param.grad` at each step.

There are however other extensions of stochastic gradient descent that compute the per-step updates using more complicated functions of gradients (some take into account the gradients' values at earlier steps).

As always, many of those are already implemented in torch. In this example we are going to use `torch.optim.Adam`.

In [ ]:
learning_rate = 0.0005
# When creating an istance of the optimizer you have to tell
# it which parameters you want to optimize. We are doing so by
# passing model.parameters() to it:
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

Once an optimizer is created we can use it by first calculating the loss on some batch of data, computing the gradients and then simply calling to `optimizer.step()`. Then repeat with the next batch and so on.

### Build pipeline

Ideally we would like to loop through all of our train data. One such loop is called an 'epoch'. We'd also like the data to be shuffled at each epoch.

Let's write a function that gives us batches of data and which we'll call at the start of each epoch:

We'll add one more utility to keep track of loss values and plot them at each epoch:

In [ ]:
from IPython.display import clear_output

class Logger:
    def __init__(self):
        self.train_loss_batch = []
        self.train_loss_epoch = []
        self.test_loss_batch = []
        self.test_loss_epoch = []
        self.train_batches_per_epoch = 0
        self.test_batches_per_epoch = 0
        self.epoch_counter = 0

    def fill_train(self, loss):
        self.train_loss_batch.append(loss)
        self.train_batches_per_epoch += 1

    def fill_test(self, loss):
        self.test_loss_batch.append(loss)
        self.test_batches_per_epoch += 1

    def finish_epoch(self):
        self.train_loss_epoch.append(np.mean(
            self.train_loss_batch[-self.train_batches_per_epoch:]
        ))
        self.test_loss_epoch.append(np.mean(
            self.test_loss_batch[-self.test_batches_per_epoch:]
        ))
        self.train_batches_per_epoch = 0
        self.test_batches_per_epoch = 0

        clear_output()

        print("epoch #{} \t train_loss: {:.8} \t test_loss: {:.8}".format(
                  self.epoch_counter,
                  self.train_loss_epoch[-1],
                  self.test_loss_epoch [-1]
              ))

        self.epoch_counter += 1

        plt.figure(figsize=(11, 5))

        plt.subplot(1, 2, 1)
        plt.plot(self.train_loss_batch, label='train loss')
        plt.xlabel('# batch iteration')
        plt.ylabel('loss')
        plt.legend()

        plt.subplot(1, 2, 2)
        plt.plot(self.train_loss_epoch, label='average train loss')
        plt.plot(self.test_loss_epoch , label='average test loss' )
        plt.legend()
        plt.xlabel('# epoch')
        plt.ylabel('loss')
        plt.show();

In [ ]:
class Model(torch.nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()
        self.m =  torch.nn.Linear(input_dim, output_dim)
        
    def forward(self, X):
        return self.m(X)
    
def train(model, optimizer, dl_train, dl_test, criterion, n_epochs):
    logger = Logger()
    
    for i_epoch in range(n_epochs):
        model.train()
        for batch_X, batch_y in dl_train:
            optimizer.zero_grad()
            
            loss = criterion(model(batch_X), batch_y)
            loss.backward()
            optimizer.step()

            logger.fill_train(loss.item())
            
        model.eval()
        with torch.no_grad():
            for batch_X, batch_y  in dl_test:
                loss = criterion(model(batch_X), batch_y)
                logger.fill_test(loss.item())

        logger.finish_epoch()
        
def predict(model, dl_test):
    model.eval()
    prediction = torch.zeros((len(dl_test.dataset), ), dtype=torch.long).cuda()
    idx = 0
    with torch.no_grad():
        for batch_X , _ in dl_test:
            pred = model(batch_X).squeeze()
            size = pred.shape[0]
            prediction[idx:idx + size] = torch.argmax(pred, dim=1)
            idx += size
    
    return prediction

### Putting it all together

Here's the setting up part, copy-pasted from previous cells:

In [ ]:
# Defining the loss function:
criterion = torch.nn.CrossEntropyLoss()

# Defining the model
input_dim = 28 * 28 # number of pixels per image
output_dim = 10 # number of classes
model = Model(input_dim, output_dim).cuda()

# Setting up the optimizer
learning_rate = 0.0005
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

train(model, optimizer, dl_train, dl_test, criterion, n_epochs = 15)

Now it's your turn to code the learning process:

In [ ]:
def accuracy_score(y_pred, y_test):
    return (y_pred == y_test).sum()/len(y_test)

In [ ]:
accuracy_score(predict(model, dl_test), y_test)

### Bonus part

Ok, so what we did so far is just a simple linear model. Let's add a hidden layer to turn it into a 'real' neural net. Modify the code above.

You can stack layers by using `torch.nn.Sequential` class:

```
model = torch.nn.Sequential(
  torch.nn.Linear(...),
  torch.nn.ReLU(),
  torch.nn.Linear(...),
)
```

In this example we've put an activation layer `torch.nn.ReLU` between the two linear ones. What will happen if we don't do that?

In [ ]:
class Model(torch.nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()
        self.m = torch.nn.Sequential(
            torch.nn.Linear(input_dim, input_dim//4),
            torch.nn.ReLU(),
            torch.nn.Linear(input_dim//4, input_dim//8),
            torch.nn.ReLU(),
            torch.nn.Linear(input_dim//8, output_dim)
        )
        
    def forward(self, X):
        return self.m(X)

In [ ]:
# Defining the loss function:
criterion = torch.nn.CrossEntropyLoss()

# Defining the model
input_dim = 28 * 28 # number of pixels per image
output_dim = 10 # number of classes

model = Model(input_dim, output_dim).cuda()

# Setting up the optimizer
learning_rate = 0.0005
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

train(model, optimizer, dl_train, dl_test, criterion, n_epochs = 15)

In [ ]:
accuracy_score(predict(model, dl_test), y_test)

## Tomorrow: Convolution operation
In case you're already intrested:
- [Pytorch Conv2d](https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html)
- [About convolution](https://cs231n.github.io/convolutional-networks/)